In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
from datetime import datetime

This code combines all the concepts introduced in Chapter 9, namely Modular functions, name scopes, Optimization,
Saving and Restoring models.


In [2]:
n_inputs = 28 * 28 
n_hidden_1 = 300
n_hidden_2 = 100
n_outputs = 10

In [3]:
def neuron_layer(X, n_neuros, name, activation):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neuros), stddev=stddev)
        W = tf.Variable(init, name = 'weights')
        b = tf.Variable(tf.zeros([n_neuros]), name = 'biases')
        z = tf.matmul(X, W) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        else:
            return z
        

In [4]:
# Construction phase
X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(dtype=tf.int64, shape=(None), name='y')

with tf.name_scope('dnn'):
    hidden_1 = neuron_layer(X, n_hidden_1, "hidden_1", activation='relu')
    hidden_2 = neuron_layer(hidden_1, n_hidden_2, "hidden_2", activation='relu')
    logits = neuron_layer(hidden_2, n_outputs, "outputs", activation = None)

with tf.name_scope('loss'):
    x_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(x_entropy, name='loss')
    
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


In [5]:
now = datetime.utcnow().strftime("%Y-%m-%d-%H:%M:%S")
root_logdir = "tf_logs"
logdir = os.path.join(os.getcwd(),"{}/run-{}".format(root_logdir, now))

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [6]:
n_epochs = 10
batch_size = 50

In [7]:
mnist = input_data.read_data_sets("tmp/data")

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
accuracy_summary = tf.summary.scalar('Accuracy', accuracy)


In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

sess = tf.InteractiveSession()
init.run()

for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
    acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
    file_writer.add_summary(accuracy_summary.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels}),epoch)
    
    print(epoch, "Train accuracy ", acc_train, "Test accuracy ", acc_test)

save_path = saver.save(sess, os.path.join(os.getcwd(),"Models/PlainTF_Model.ckpt"))
 

0 Train accuracy  0.84 Test accuracy  0.9111
1 Train accuracy  0.9 Test accuracy  0.9307
2 Train accuracy  0.92 Test accuracy  0.9391
3 Train accuracy  1.0 Test accuracy  0.9453
4 Train accuracy  0.94 Test accuracy  0.9496
5 Train accuracy  0.98 Test accuracy  0.9523
6 Train accuracy  0.94 Test accuracy  0.9565
7 Train accuracy  0.98 Test accuracy  0.9582
8 Train accuracy  0.92 Test accuracy  0.9603
9 Train accuracy  0.94 Test accuracy  0.9631


In [10]:
sess.close()
file_writer.close()